# Esempio Completo: Generazione e Analisi Tracce

Questo notebook mostra come:
1. Generare tutte le possibili tracce da un modello BPMN
2. Analizzare le tracce generate
3. Visualizzare i risultati

## Prerequisiti

Assicurati che i server siano in esecuzione:
- Simulatore: `cd simulator && python src/main.py`
- PACO: `python -m src.server`

## 1. Setup

In [1]:
import json
import subprocess
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configura stile grafici
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

ModuleNotFoundError: No module named 'seaborn'

## 2. Carica un Modello BPMN

Usiamo un esempio esistente

In [ ]:
bpmn_file = "simulator\tests\spin\output\choice_containing_nature.json"

with open(bpmn_file, 'r') as f:
    bpmn = json.load(f)

print(f"Modello BPMN caricato: {bpmn.get('expression', '')}")
print(f"Dimensioni impatto: {bpmn.get('impacts_names', [])}")

: 

: 

: 

## 3. Genera Tutte le Tracce

Usa lo script `generate_all_traces.py` per generare tutte le possibili tracce

In [ ]:
# Nome file output
traces_file = "example_traces.json"
execution_tree_file = "example_execution_tree.json"

# Esegui lo script di generazione
result = subprocess.run([
    "python", "generate_all_traces.py", 
    bpmn_file,
    "-o", traces_file,
    "--execution-tree-output", execution_tree_file,
    "--max-nodes", "1000"
], capture_output=True, text=True)

print(result.stdout)
if result.returncode != 0:
    print("Errore:")
    print(result.stderr)

## 4. Carica le Tracce Generate

In [ ]:
with open(traces_file, 'r') as f:
    traces_data = json.load(f)

traces = traces_data['traces']
impacts_names = traces_data['bpmn']['impacts_names']

print(f"Numero di tracce: {len(traces)}")
print(f"Dimensioni impatto: {impacts_names}")

## 5. Converti in DataFrame per Analisi

In [ ]:
# Crea DataFrame
df_data = []
for trace in traces:
    row = {
        'id': trace['id'],
        'decisions': ' -> '.join(trace['decisions']),
        'probability': trace['probability'],
        'execution_time': trace['execution_time'],
        'num_decisions': len(trace['decisions'])
    }
    # Aggiungi impatti
    for impact_name, value in trace['impacts'].items():
        row[impact_name] = value
    df_data.append(row)

df = pd.DataFrame(df_data)
df.head()

## 6. Statistiche Descrittive

In [ ]:
print("=" * 60)
print("STATISTICHE DESCRITTIVE")
print("=" * 60)
print()

# Statistiche su impatti
print("Impatti:")
for impact_name in impacts_names:
    if impact_name in df.columns:
        print(f"\n{impact_name}:")
        print(f"  Media: {df[impact_name].mean():.2f}")
        print(f"  Mediana: {df[impact_name].median():.2f}")
        print(f"  Min: {df[impact_name].min():.2f}")
        print(f"  Max: {df[impact_name].max():.2f}")
        print(f"  Std Dev: {df[impact_name].std():.2f}")

# Statistiche su probabilità
print(f"\nProbabilità:")
print(f"  Totale: {df['probability'].sum():.4f}")
print(f"  Media: {df['probability'].mean():.4f}")
print(f"  Min: {df['probability'].min():.4f}")
print(f"  Max: {df['probability'].max():.4f}")

# Statistiche su tempo di esecuzione
print(f"\nTempo di esecuzione:")
print(f"  Media: {df['execution_time'].mean():.2f}")
print(f"  Min: {df['execution_time'].min():.2f}")
print(f"  Max: {df['execution_time'].max():.2f}")

## 7. Visualizzazioni

### 7.1 Distribuzione degli Impatti

In [ ]:
fig, axes = plt.subplots(1, len(impacts_names), figsize=(6*len(impacts_names), 5))

if len(impacts_names) == 1:
    axes = [axes]

for i, impact_name in enumerate(impacts_names):
    if impact_name in df.columns:
        axes[i].hist(df[impact_name], bins=20, edgecolor='black')
        axes[i].set_xlabel(impact_name)
        axes[i].set_ylabel('Frequenza')
        axes[i].set_title(f'Distribuzione {impact_name}')
        axes[i].axvline(df[impact_name].mean(), color='r', linestyle='--', label='Media')
        axes[i].legend()

plt.tight_layout()
plt.show()

### 7.2 Scatter Plot degli Impatti (se ci sono almeno 2 dimensioni)

In [ ]:
if len(impacts_names) >= 2:
    plt.figure(figsize=(10, 8))
    
    # Scatter plot
    scatter = plt.scatter(
        df[impacts_names[0]], 
        df[impacts_names[1]],
        c=df['probability'],
        cmap='viridis',
        s=100,
        alpha=0.6,
        edgecolors='black'
    )
    
    plt.colorbar(scatter, label='Probabilità')
    plt.xlabel(impacts_names[0])
    plt.ylabel(impacts_names[1])
    plt.title(f'{impacts_names[0]} vs {impacts_names[1]}')
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("Serve almeno 2 dimensioni di impatto per lo scatter plot")

### 7.3 Frontiera di Pareto

In [ ]:
def is_pareto_efficient(costs):
    """
    Trova i punti sulla frontiera di Pareto.
    Assume che valori più bassi siano migliori.
    """
    is_efficient = np.ones(costs.shape[0], dtype=bool)
    for i, c in enumerate(costs):
        if is_efficient[i]:
            # Rimuovi punti dominati
            is_efficient[is_efficient] = np.any(costs[is_efficient] < c, axis=1)
            is_efficient[i] = True
    return is_efficient

if len(impacts_names) >= 2:
    import numpy as np
    
    # Crea matrice dei costi
    costs = df[impacts_names].values
    
    # Trova punti Pareto-efficienti
    pareto_mask = is_pareto_efficient(costs)
    pareto_df = df[pareto_mask]
    
    print(f"Tracce sulla frontiera di Pareto: {len(pareto_df)} su {len(df)}")
    
    # Visualizza
    plt.figure(figsize=(10, 8))
    
    # Tracce normali
    plt.scatter(
        df[~pareto_mask][impacts_names[0]], 
        df[~pareto_mask][impacts_names[1]],
        c='lightgray',
        s=50,
        alpha=0.5,
        label='Tracce dominate'
    )
    
    # Tracce Pareto-efficienti
    plt.scatter(
        pareto_df[impacts_names[0]], 
        pareto_df[impacts_names[1]],
        c='red',
        s=150,
        marker='*',
        edgecolors='black',
        label='Frontiera di Pareto',
        zorder=5
    )
    
    plt.xlabel(impacts_names[0])
    plt.ylabel(impacts_names[1])
    plt.title('Frontiera di Pareto')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # Mostra le tracce Pareto-efficienti
    print("\nTracce Pareto-efficienti:")
    display(pareto_df[['decisions'] + impacts_names + ['probability', 'execution_time']])
else:
    print("Serve almeno 2 dimensioni di impatto per la frontiera di Pareto")

## 8. Analisi per Decisione

In [ ]:
from collections import defaultdict

# Raggruppa tracce per decisioni
decision_impact = defaultdict(list)

for _, row in df.iterrows():
    decisions = row['decisions'].split(' -> ')
    for decision in decisions:
        for impact_name in impacts_names:
            decision_impact[decision].append({
                'impact': impact_name,
                'value': row[impact_name]
            })

# Visualizza impatto medio per decisione
decision_summary = []
for decision, impacts in decision_impact.items():
    # Raggruppa per tipo di impatto
    by_impact = defaultdict(list)
    for item in impacts:
        by_impact[item['impact']].append(item['value'])
    
    row = {'decision': decision}
    for impact_name, values in by_impact.items():
        row[f'{impact_name}_avg'] = np.mean(values)
        row[f'{impact_name}_count'] = len(values)
    
    decision_summary.append(row)

decision_df = pd.DataFrame(decision_summary)
print("Impatto medio per decisione:")
display(decision_df)

## 9. Esporta Risultati

In [ ]:
# Esporta DataFrame completo
df.to_csv('traces_analysis.csv', index=False)
print("✓ DataFrame salvato in traces_analysis.csv")

# Esporta tracce Pareto-efficienti (se esistono)
if len(impacts_names) >= 2 and 'pareto_df' in locals():
    pareto_df.to_csv('pareto_traces.csv', index=False)
    print("✓ Tracce Pareto-efficienti salvate in pareto_traces.csv")

# Esporta riepilogo decisioni
if 'decision_df' in locals():
    decision_df.to_csv('decision_impact.csv', index=False)
    print("✓ Impatto decisioni salvato in decision_impact.csv")

## 10. Conclusioni

Questo notebook ha mostrato come:
- Generare tutte le tracce possibili da un modello BPMN
- Analizzare le tracce con statistiche descrittive
- Visualizzare la distribuzione degli impatti
- Identificare la frontiera di Pareto
- Analizzare l'impatto di singole decisioni
- Esportare i risultati per ulteriori analisi

### Prossimi Passi

- Confrontare diverse strategie di esecuzione
- Ottimizzare le decisioni per obiettivi specifici
- Analizzare la sensibilità ai parametri del modello
- Usare tecniche di machine learning per prevedere gli esiti